### Question 6: Least frequent pickup location zone

Load the zone lookup data into a temp view in Spark:

```bash
wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
```

Using the zone lookup data and the Yellow October 2024 data, what is the name of the LEAST frequent pickup location Zone?

- Governor's Island/Ellis Island/Liberty Island
- Arden Heights
- Rikers Island
- Jamaica Bay


In [4]:
!wget wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2025-03-07 16:10:51--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2025-03-07 16:10:53--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.164.82.160, 3.164.82.112, 3.164.82.197, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.164.82.160|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0.01s   

2025-03-07 16:10:54 (820 KB/s) - ‘taxi_zone_lookup.csv’ saved [12331/12331]

FINISHED --2025-03-07 16:10:54--
Total wall clock time: 2.7s
Downloaded: 1 files, 12K in 0.01s (820 KB/s)


In [5]:
import pyspark
from pyspark.sql import SparkSession, types

In [6]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName('test') \
        .getOrCreate()

25/03/07 16:11:07 WARN Utils: Your hostname, SOLIDCAD-SERVER resolves to a loopback address: 127.0.1.1; using 192.168.160.37 instead (on interface eth0)
25/03/07 16:11:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/07 16:11:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/07 16:11:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/03/07 16:11:10 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/03/07 16:11:10 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [7]:
df_yellow = spark.read \
     .option("header", "true") \
     .parquet('/home/myothet/repos/data-engineering/batch/yellow_tripdata_2024-10.parquet')

In [8]:
df_zones = spark.read \
     .option("header", "true") \
     .csv('/home/myothet/repos/data-engineering/batch/taxi_zone_lookup.csv')

In [9]:
df_yellow.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [10]:
df_zones.printSchema()

root
 |-- LocationID: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [11]:
df_zones.schema

StructType([StructField('LocationID', StringType(), True), StructField('Borough', StringType(), True), StructField('Zone', StringType(), True), StructField('service_zone', StringType(), True)])

In [12]:
zones_schema = types.StructType([
    types.StructField('LocationID', types.IntegerType(), True), 
    types.StructField('Borough', types.StringType(), True), 
    types.StructField('Zone', types.StringType(), True), 
    types.StructField('service_zone', types.StringType(), True)])

In [14]:
df_zones = spark.read \
     .option("header", "true") \
     .schema(zones_schema) \
     .csv('/home/myothet/repos/data-engineering/batch/taxi_zone_lookup.csv')

In [18]:
df_zones.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [19]:
df_result = df_yellow.join(df_zones, df_yellow.PULocationID == df_zones.LocationID)

In [20]:
df_result.count()

3833771

In [24]:
df_result.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+----------+---------+-------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|LocationID|  Borough|               Zone|service_zone|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+----------+---------+-------------------+------------+
|       2| 2024-10-01 00:

In [26]:
df_result.createOrReplaceTempView ('df_joined')

In [33]:
query = """
SELECT
    PULocationID,
    COUNT(1)
FROM 
    df_joined
GROUP BY
    1
ORDER BY
    2 ASC
LIMIT 5;
"""

In [34]:
spark.sql(query).show()

+------------+--------+
|PULocationID|count(1)|
+------------+--------+
|         105|       1|
|           5|       2|
|         199|       2|
|           2|       3|
|         111|       3|
+------------+--------+



In [35]:
df_zones.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [39]:
df_zones \
  .select('LocationID', 'Borough', 'Zone') \
  .filter("LocationID = '105'") \
  .take(1)

[Row(LocationID=105, Borough='Manhattan', Zone="Governor's Island/Ellis Island/Liberty Island")]